In [3]:
import io
import itertools

import tensorflow as tf
from tensorflow import keras

import matplotlib.pyplot as plt
import numpy as np
import sklearn.metrics

In [14]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
# Load the TensorBoard notebook extension.
%load_ext tensorboard

from tensorboard import notebook
import os
from datetime import datetime
import shlex

def kill_all_tensorboard():
    """
    détruit tous les serveurs tensorboard
    """
    import tempfile
    path = os.path.join(tempfile.gettempdir(), ".tensorboard-info")
    for i in os.listdir(path) :
        os.unlink(path+'\\'+i)
    os.system("taskkill /F /IM tensorboard.exe")
    
def kill_pid_tensorboard(pid):
    """
    détruit le tensorboards correspondant au pid
    """
    
    import tempfile
    try:
        path = os.path.join(tempfile.gettempdir(), ".tensorboard-info")
        os.unlink(path+'\\pid-'+str(pid)+".info")
    except:
        print("Fichier introuvable")
    os.system("taskkill /F /PID "+str(pid))
    
def kill_port_tensorboard(port):
    """
    détruit les tensorboards utilisant le port indiqué
    """
    occupied = port_occupied(port)
    for i in port_occupied(port):
        print("port {} occupé, kill pid {}".format(port, i.pid))
        kill_pid_tensorboard(i.pid)
    
def port_occupied(port):
    """
    Retourne les tensorboard qui utilise le port indiqué
    """
    l = []
    for i in notebook.manager.get_all():
        if i.port == port:
            l += [i]
    return l

def launch_tensorboard(log_file, port = 6006):
    """
    Contruit un serveur tensorboard au port indiqué à partir de log_file.
    Si le port est déjà pris, détruit les anciens serveurs 
    """
    kill_port_tensorboard(port)
    parsed_args = shlex.split("--logdir {} --port {}".format(log_file,port), comments=True, posix=True)
    start_result = notebook.manager.start(parsed_args, timeout=datetime.timedelta(seconds=10))
    notebook.display(port)
    
def affiche_port(port):
    """
    affiche le tensorboard du port indiqué
    """
    notebook.display(port)
    
def affiche_all_tensorboard():
    """
    affiche les serveurs tensorboard actifs
    """
    l = notebook.manager.get_all()
    for i in l:
        print("pid : {} | port : {} | logdir : {}".format(i.pid, i.port, i.logdir))


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [15]:
affiche_all_tensorboard()

pid : 6924 | port : 8008 | logdir : logs/train_data


In [6]:
affiche_port(6006)

In [7]:
l = notebook.manager.get_all()

In [8]:
# Download the data. The data is already divided into train and test.
# The labels are integers representing classes.
fashion_mnist = keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = \
    fashion_mnist.load_data()

# Names of the integer classes, i.e., 0 -> T-short/top, 1 -> Trouser, etc.
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 
    'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']


In [9]:
print("Shape: ", train_images[0].shape)
print("Label: ", train_labels[0], "->", class_names[train_labels[0]])


Shape:  (28, 28)
Label:  9 -> Ankle boot


In [10]:
# Reshape the image for the Summary API.
img = np.reshape(train_images[0], (-1, 28, 28, 1))


In [11]:
# Clear out any prior log data.
!rm -rf logs

# Sets up a timestamped log directory.
logdir = "logs/train_data/" + datetime.now().strftime("%Y%m%d-%H%M%S")
# Creates a file writer for the log directory.
file_writer = tf.summary.create_file_writer(logdir)

# Using the file writer, log the reshaped image.
with file_writer.as_default():
  tf.summary.image("Training data", img, step=0)

'rm' n'est pas reconnu en tant que commande interne
ou externe, un programme ex‚cutable ou un fichier de commandes.


In [12]:
%tensorboard --logdir logs/train_data

ERROR: Timed out waiting for TensorBoard to start. It may still be running as pid 10256.

In [16]:
kill_pid_tensorboard(13088)

In [17]:
notebook.list()

No known TensorBoard instances running.


In [41]:
kill_all_tensorboard()

In [53]:
notebook.manager.get_all()[0].pid

5700

In [67]:
launch_tensorboard('logs/train_data', port = 8008)

port 8008 occupé, kill pid 1472
Selecting TensorBoard with logdir logs/train_data (started 0:00:07 ago; port 8008, pid 6924).


In [32]:
with file_writer.as_default():
  # Don't forget to reshape.
  images = np.reshape(train_images[0:25], (-1, 28, 28, 1))
  tf.summary.image("25 training data examples", images, max_outputs=25, step=0)



Selecting TensorBoard with logdir logs/train_data (started 0:00:07 ago; port 6006, pid 7548).


In [33]:
notebook.list()

Known TensorBoard instances:
  - port 6006: logdir logs/train_data (started 0:00:21 ago; pid 7548)
